This script is a single-file PyTorch experiment runner for stress prediction. It:

Builds sliding windows of participant data with sizes [3,4,5,6,7,10,14] to predict next-day stress levels.

Creates inputs for ANN and sequence models (CNN1D, RNN, GRU, LSTM).

Splits participants train/validation/test to avoid data leakage.

Trains models with class-weighted cross-entropy loss, records metrics (loss, accuracy, F1), and applies early stopping based on validation F1.

Saves results summaries, confusion matrices, and plots to the outputs directory.

Supports evaluation across multiple window sizes and models, providing detailed per-class metrics and sample counts.

In [ ]:
"""
experiment_runner.py

Single-file PyTorch experiment runner that:
- builds sliding windows for window_sizes = [3,4,5,6,7,10,14]
- creates ANN and sequence inputs
- trains ANN, CNN1D, SimpleRNN, GRU, LSTM
- participant-wise split (train/val/test)
- saves results to CSV and confusion matrix PNGs

Usage:
    python experiment_runner.py
"""

import os
import math
import copy
import random
from pathlib import Path
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, precision_recall_fscore_support, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# -------------------------
# Config
# -------------------------
CSV_PATH = "stress_detection.csv"
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
WINDOW_SIZES = [3,4,5,6,7,10,14]
RAW_FEATURES = [
    'Openness','Conscientiousness','Extraversion','Agreeableness','Neuroticism',
    'sleep_time','wake_time','sleep_duration','PSQI_score',
    'call_duration','num_calls','num_sms',
    'screen_on_time','skin_conductance','accelerometer',
    'mobility_radius','mobility_distance'
]
STATIC_PERSONALITY = ['Openness','Conscientiousness','Extraversion','Agreeableness','Neuroticism']
TARGET_COL = 'PSS_score'
CLASS_COL = 'stress_class'  # we will add

# train/val/test by participant fractions
TRAIN_P = 0.7
VAL_P = 0.15
TEST_P = 0.15

# training hyperparams (conservative)
EPOCHS = 60
BATCH_SIZE = 64
LR = 1e-3
PATIENCE = 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# models to run
MODEL_NAMES = ["ANN","CNN1D","RNN","GRU","LSTM"]

# -------------------------
# Helpers
# -------------------------
def stress_to_class(score):
    # as user specified: 0-13 low, 14-26 moderate, 27-40 high
    s = float(score)
    if s <= 13: return 0
    if s <= 26: return 1
    return 2

def ensure_dir(path):
    Path(path).mkdir(parents=True, exist_ok=True)

# -------------------------
# Dataset builder
# -------------------------
def build_windows(df, raw_features, window_size=4):
    """
    Returns:
      X_seq: list of arrays shape (window_size, F)
      X_flat: list of 1D arrays flattened + aggregated features
      y: list of int labels
      participants: list of participant ids (aligned with samples)
    """
    X_seq = []
    X_flat = []
    y = []
    participants = []

    for pid, df_p in df.groupby("participant_id"):
        df_p = df_p.sort_values("day").reset_index(drop=True)
        n = len(df_p)
        # skip participants with insufficient rows
        if n <= window_size:
            continue
        arr = df_p[raw_features].values.astype(float)
        for i in range(n - window_size):
            window = arr[i:i+window_size]  # shape (window_size, F)
            # aggregated features
            mean_vals = window.mean(axis=0)
            std_vals = window.std(axis=0)
            min_vals = window.min(axis=0)
            max_vals = window.max(axis=0)
            slope_vals = window[-1] - window[0]
            agg = np.concatenate([mean_vals, std_vals, min_vals, max_vals, slope_vals])

            flat = np.concatenate([window.flatten(), agg])
            target_score = df_p[TARGET_COL].iloc[i+window_size]  # next day
            cls = stress_to_class(target_score)

            X_seq.append(window.copy())
            X_flat.append(flat.copy())
            y.append(int(cls))
            participants.append(pid)
    return np.array(X_seq), np.array(X_flat), np.array(y), np.array(participants)


# -------------------------
# PyTorch Dataset
# -------------------------
class StressDataset(Dataset):
    def __init__(self, X, y):
        # X: either (N,window,F) for seq models or (N,d) for ANN
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# -------------------------
# Models
# -------------------------
class ANNModel(nn.Module):
    def __init__(self, in_dim, hidden=[256,128,64], num_classes=3, p_drop=0.3):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers.append(nn.Linear(last,h))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(h))
            layers.append(nn.Dropout(p_drop))
            last = h
        layers.append(nn.Linear(last,num_classes))
        self.net = nn.Sequential(*layers)
    def forward(self,x):
        return self.net(x)

class CNN1DModel(nn.Module):
    def __init__(self, seq_len, feat_dim, num_classes=3):
        super().__init__()
        # treat features as channels -> input shape (B, F, L)
        self.conv1 = nn.Conv1d(feat_dim, 64, kernel_size=2)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64,128,kernel_size=2)
        self.bn2 = nn.BatchNorm1d(128)
        # compute flattened size after convs:
        l = seq_len - (2-1) - (2-1)  # two convs kernel 2 valid padding
        if l < 1: l = 1
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*1,64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64,num_classes)
        )
    def forward(self,x):
        # x: (B, L, F)
        x = x.permute(0,2,1)  # -> (B, F, L)
        x = self.conv1(x); x = self.bn1(x); x = torch.relu(x)
        x = self.conv2(x); x = self.bn2(x); x = torch.relu(x)
        x = self.gap(x)  # (B,128,1)
        return self.fc(x)

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden=64, num_layers=1, rnn_type='RNN', num_classes=3, dropout=0.2):
        super().__init__()
        if rnn_type=='RNN':
            self.rnn = nn.RNN(input_dim,hidden,num_layers,batch_first=True,nonlinearity='tanh',dropout=dropout)
        elif rnn_type=='GRU':
            self.rnn = nn.GRU(input_dim,hidden,num_layers,batch_first=True,dropout=dropout)
        elif rnn_type=='LSTM':
            self.rnn = nn.LSTM(input_dim,hidden,num_layers,batch_first=True,dropout=dropout)
        else:
            raise ValueError("Unknown rnn_type")
        self.head = nn.Sequential(
            nn.Linear(hidden,32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32,num_classes)
        )
        self.rnn_type = rnn_type
    def forward(self,x):
        # x: (B,L,F)
        out, _ = self.rnn(x)  # out: (B,L,H)
        last = out[:, -1, :]  # last timestep
        return self.head(last)

# -------------------------
# Training & evaluation utilities
# -------------------------
def compute_class_weights(y):
    counts = Counter(y.tolist())
    total = sum(counts.values())
    num_classes = len(counts)
    weights = []
    for i in range(max(counts.keys())+1):
        cnt = counts.get(i, 0)
        if cnt==0:
            weights.append(0.0)
        else:
            weights.append(total / (num_classes * cnt))
    return torch.tensor(weights, dtype=torch.float32, device=DEVICE)

def train_one_epoch(model, loader, opt, criterion):
    model.train()
    total_loss = 0.0
    preds = []
    trues = []
    for xb, yb in loader:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)
        out = model(xb)
        loss = criterion(out, yb)
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item() * xb.size(0)
        preds.append(out.detach().argmax(dim=1).cpu().numpy())
        trues.append(yb.cpu().numpy())
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mean_loss = total_loss / len(trues)
    acc = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average='macro', zero_division=0)
    return mean_loss, acc, f1

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    preds = []
    trues = []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            out = model(xb)
            loss = criterion(out, yb)
            total_loss += loss.item() * xb.size(0)
            preds.append(out.argmax(dim=1).cpu().numpy())
            trues.append(yb.cpu().numpy())
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mean_loss = total_loss / len(trues)
    acc = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average='macro', zero_division=0)
    prec, rec, f1s, sup = precision_recall_fscore_support(trues, preds, average=None, zero_division=0)
    return mean_loss, acc, f1, prec, rec, f1s, sup, preds, trues

# -------------------------
# Experiment loop
# -------------------------
def run_experiment_for_window(df, window_size, model_name, train_pids, val_pids, test_pids):
    # build windows
    X_seq, X_flat, y, pids = build_windows(df, RAW_FEATURES, window_size=window_size)
    if len(y)==0:
        print(f"No samples for window {window_size}")
        return None

    # split by given participant ids - keep only samples whose pid in sets
    train_mask = np.isin(pids, train_pids)
    val_mask = np.isin(pids, val_pids)
    test_mask = np.isin(pids, test_pids)

    X_seq_train, X_seq_val, X_seq_test = X_seq[train_mask], X_seq[val_mask], X_seq[test_mask]
    X_flat_train, X_flat_val, X_flat_test = X_flat[train_mask], X_flat[val_mask], X_flat[test_mask]
    y_train, y_val, y_test = y[train_mask], y[val_mask], y[test_mask]

    # skip if any split empty
    if len(y_train)==0 or len(y_val)==0 or len(y_test)==0:
        print("One split empty for window", window_size)
        return None

    # Choose input style based on model
    if model_name=="ANN":
        # scale flattened features
        scaler = StandardScaler().fit(X_flat_train)
        Xtr = scaler.transform(X_flat_train)
        Xv = scaler.transform(X_flat_val)
        Xt = scaler.transform(X_flat_test)
        input_dim = Xtr.shape[1]
        train_ds = StressDataset(Xtr, y_train)
        val_ds = StressDataset(Xv, y_val)
        test_ds = StressDataset(Xt, y_test)
    else:
        # sequence models: use X_seq shaped (N, L, F) - scale per feature across training sequence samples
        # reshape to (N*L, F) to fit scaler
        Ntr, L, F = X_seq_train.shape
        scaler = StandardScaler().fit(X_seq_train.reshape(-1,F))
        Xtr = scaler.transform(X_seq_train.reshape(-1,F)).reshape(Ntr,L,F)
        Xv = scaler.transform(X_seq_val.reshape(-1,F)).reshape(X_seq_val.shape)
        Xt = scaler.transform(X_seq_test.reshape(-1,F)).reshape(X_seq_test.shape)
        train_ds = StressDataset(Xtr, y_train)
        val_ds = StressDataset(Xv, y_val)
        test_ds = StressDataset(Xt, y_test)
        input_dim = F

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    # prepare model
    if model_name=="ANN":
        model = ANNModel(input_dim).to(DEVICE)
    elif model_name=="CNN1D":
        # seq len and feat_dim
        seq_len = X_seq_train.shape[1]
        feat_dim = X_seq_train.shape[2]
        model = CNN1DModel(seq_len, feat_dim).to(DEVICE)
    elif model_name in ("RNN","GRU","LSTM"):
        seq_len = X_seq_train.shape[1]
        feat_dim = X_seq_train.shape[2]
        model = RNNModel(feat_dim, hidden=64, num_layers=1, rnn_type=model_name, num_classes=3).to(DEVICE)
    else:
        raise ValueError("Unknown model")

    # class weights
    class_weights = compute_class_weights(torch.tensor(y_train))
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)

    # training loop with early stopping on val f1
    best_val_f1 = -1.0
    best_state = None
    patience = PATIENCE
    cur_wait = 0

    for epoch in range(1, EPOCHS+1):
        train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, opt, criterion)
        val_loss, val_acc, val_f1, *_ = evaluate(model, val_loader, criterion)

        # print progress
        print(f"[W{window_size}] {model_name} Epoch {epoch:02d} | train_loss {train_loss:.4f} acc {train_acc:.3f} f1 {train_f1:.3f} || val_loss {val_loss:.4f} acc {val_acc:.3f} f1 {val_f1:.3f}")

        if val_f1 > best_val_f1 + 1e-4:
            best_val_f1 = val_f1
            best_state = copy.deepcopy(model.state_dict())
            cur_wait = 0
        else:
            cur_wait += 1
            if cur_wait >= patience:
                print("Early stopping.")
                break

    # load best and evaluate on test
    if best_state is not None:
        model.load_state_dict(best_state)

    test_loss, test_acc, test_f1, prec, rec, f1s, sup, preds, trues = evaluate(model, test_loader, criterion)

    # save confusion matrix
    cm = confusion_matrix(trues, preds)
    cm_path = OUTPUT_DIR / f"cm_w{window_size}_{model_name}.npy"
    np.save(cm_path, cm)

    # also save a small png
    plt.figure(figsize=(4,4))
    plt.imshow(cm, cmap='Blues')
    plt.title(f"CM W{window_size} {model_name}")
    plt.colorbar()
    plt.xlabel("pred")
    plt.ylabel("true")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / f"cm_w{window_size}_{model_name}.png")
    plt.close()

    # return summary dict
    summary = {
        "window": window_size,
        "model": model_name,
        "train_samples": len(y_train),
        "val_samples": len(y_val),
        "test_samples": len(y_test),
        "test_loss": float(test_loss),
        "test_acc": float(test_acc),
        "test_macro_f1": float(test_f1),
        "per_class_prec": prec.tolist(),
        "per_class_rec": rec.tolist(),
        "per_class_f1": f1s.tolist(),
        "support": sup.tolist()
    }
    return summary

# -------------------------
# Main orchestration
# -------------------------
def main():
    print("Device:", DEVICE)
    df = pd.read_csv(CSV_PATH)
    # ensure ordering and class column
    df = df.sort_values(['participant_id','day']).reset_index(drop=True)
    df[CLASS_COL] = df[TARGET_COL].apply(stress_to_class)

    # participants split
    pids = df['participant_id'].unique()
    random.shuffle(pids)
    n = len(pids)
    n_train = int(n * TRAIN_P)
    n_val = int(n * VAL_P)
    train_pids = pids[:n_train]
    val_pids = pids[n_train:n_train+n_val]
    test_pids = pids[n_train+n_val:]
    print(f"Participants: total {n} train {len(train_pids)} val {len(val_pids)} test {len(test_pids)}")

    results = []
    for w in WINDOW_SIZES:
        for mname in MODEL_NAMES:
            print("="*60)
            print(f"Running Window={w} Model={mname}")
            try:
                res = run_experiment_for_window(df, w, mname, train_pids, val_pids, test_pids)
                if res is not None:
                    results.append(res)
                    # save intermediate results
                    pd.DataFrame(results).to_csv(OUTPUT_DIR / "results_summary.csv", index=False)
            except Exception as e:
                print("Error running", w, mname, e)

    # final save
    if results:
        out_df = pd.DataFrame(results)
        out_df.to_csv(OUTPUT_DIR / "results_summary.csv", index=False)
        print("Saved results to", OUTPUT_DIR / "results_summary.csv")
    else:
        print("No results to save.")

if __name__ == "__main__":
    main()


Device: cpu
Participants: total 100 train 70 val 15 test 15
Running Window=3 Model=ANN
[W3] ANN Epoch 01 | train_loss 1.1996 acc 0.335 f1 0.307 || val_loss 1.1341 acc 0.299 f1 0.278
[W3] ANN Epoch 02 | train_loss 1.0910 acc 0.384 f1 0.362 || val_loss 1.1551 acc 0.277 f1 0.265
[W3] ANN Epoch 03 | train_loss 1.0485 acc 0.417 f1 0.397 || val_loss 1.1741 acc 0.314 f1 0.304
[W3] ANN Epoch 04 | train_loss 0.9981 acc 0.450 f1 0.432 || val_loss 1.2088 acc 0.321 f1 0.305
[W3] ANN Epoch 05 | train_loss 0.9691 acc 0.472 f1 0.455 || val_loss 1.2198 acc 0.299 f1 0.285
[W3] ANN Epoch 06 | train_loss 0.8913 acc 0.517 f1 0.506 || val_loss 1.2316 acc 0.306 f1 0.288
[W3] ANN Epoch 07 | train_loss 0.8612 acc 0.532 f1 0.515 || val_loss 1.2500 acc 0.343 f1 0.316
[W3] ANN Epoch 08 | train_loss 0.8176 acc 0.564 f1 0.553 || val_loss 1.2950 acc 0.368 f1 0.331
[W3] ANN Epoch 09 | train_loss 0.7710 acc 0.578 f1 0.569 || val_loss 1.2601 acc 0.388 f1 0.347
[W3] ANN Epoch 10 | train_loss 0.7605 acc 0.605 f1 0.590 |

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W3] RNN Epoch 01 | train_loss 1.1046 acc 0.265 f1 0.235 || val_loss 1.0999 acc 0.316 f1 0.279
[W3] RNN Epoch 02 | train_loss 1.0923 acc 0.367 f1 0.327 || val_loss 1.1046 acc 0.323 f1 0.273
[W3] RNN Epoch 03 | train_loss 1.0822 acc 0.419 f1 0.392 || val_loss 1.1124 acc 0.363 f1 0.323
[W3] RNN Epoch 04 | train_loss 1.0748 acc 0.439 f1 0.406 || val_loss 1.1236 acc 0.321 f1 0.290
[W3] RNN Epoch 05 | train_loss 1.0675 acc 0.421 f1 0.398 || val_loss 1.1301 acc 0.321 f1 0.295
[W3] RNN Epoch 06 | train_loss 1.0599 acc 0.399 f1 0.383 || val_loss 1.1415 acc 0.311 f1 0.290
[W3] RNN Epoch 07 | train_loss 1.0464 acc 0.453 f1 0.427 || val_loss 1.1456 acc 0.321 f1 0.300
[W3] RNN Epoch 08 | train_loss 1.0444 acc 0.430 f1 0.410 || val_loss 1.1548 acc 0.316 f1 0.295
[W3] RNN Epoch 09 | train_loss 1.0281 acc 0.414 f1 0.402 || val_loss 1.1745 acc 0.323 f1 0.299
[W3] RNN Epoch 10 | train_loss 1.0220 acc 0.437 f1 0.420 || val_loss 1.1806 acc 0.316 f1 0.295
[W3] RNN Epoch 11 | train_loss 1.0159 acc 0.434 f1

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W3] GRU Epoch 01 | train_loss 1.0994 acc 0.369 f1 0.317 || val_loss 1.1000 acc 0.360 f1 0.290
[W3] GRU Epoch 02 | train_loss 1.0946 acc 0.425 f1 0.374 || val_loss 1.1006 acc 0.393 f1 0.322
[W3] GRU Epoch 03 | train_loss 1.0912 acc 0.453 f1 0.400 || val_loss 1.1030 acc 0.402 f1 0.351
[W3] GRU Epoch 04 | train_loss 1.0874 acc 0.447 f1 0.391 || val_loss 1.1050 acc 0.393 f1 0.353
[W3] GRU Epoch 05 | train_loss 1.0799 acc 0.462 f1 0.420 || val_loss 1.1098 acc 0.405 f1 0.364
[W3] GRU Epoch 06 | train_loss 1.0753 acc 0.425 f1 0.400 || val_loss 1.1118 acc 0.370 f1 0.339
[W3] GRU Epoch 07 | train_loss 1.0636 acc 0.454 f1 0.420 || val_loss 1.1243 acc 0.393 f1 0.358
[W3] GRU Epoch 08 | train_loss 1.0571 acc 0.442 f1 0.412 || val_loss 1.1313 acc 0.373 f1 0.343
[W3] GRU Epoch 09 | train_loss 1.0409 acc 0.448 f1 0.424 || val_loss 1.1409 acc 0.351 f1 0.327
[W3] GRU Epoch 10 | train_loss 1.0269 acc 0.443 f1 0.424 || val_loss 1.1567 acc 0.373 f1 0.340
[W3] GRU Epoch 11 | train_loss 1.0151 acc 0.460 f1

...

[W14] RNN Epoch 01 | train_loss 1.1051 acc 0.335 f1 0.252 || val_loss 1.0993 acc 0.283 f1 0.202
[W14] RNN Epoch 02 | train_loss 1.0920 acc 0.382 f1 0.328 || val_loss 1.1021 acc 0.304 f1 0.247
[W14] RNN Epoch 03 | train_loss 1.0838 acc 0.403 f1 0.362 || val_loss 1.1067 acc 0.338 f1 0.295
[W14] RNN Epoch 04 | train_loss 1.0735 acc 0.431 f1 0.403 || val_loss 1.1074 acc 0.321 f1 0.297
[W14] RNN Epoch 05 | train_loss 1.0655 acc 0.446 f1 0.421 || val_loss 1.1128 acc 0.329 f1 0.304
[W14] RNN Epoch 06 | train_loss 1.0486 acc 0.438 f1 0.420 || val_loss 1.1145 acc 0.333 f1 0.306
[W14] RNN Epoch 07 | train_loss 1.0345 acc 0.466 f1 0.442 || val_loss 1.1209 acc 0.333 f1 0.315
[W14] RNN Epoch 08 | train_loss 1.0086 acc 0.449 f1 0.434 || val_loss 1.1370 acc 0.350 f1 0.333
[W14] RNN Epoch 09 | train_loss 0.9885 acc 0.449 f1 0.434 || val_loss 1.1773 acc 0.375 f1 0.350
[W14] RNN Epoch 10 | train_loss 0.9689 acc 0.468 f1 0.450 || val_loss 1.2063 acc 0.375 f1 0.354
[W14] RNN Epoch 11 | train_loss 0.9488 a

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W14] GRU Epoch 01 | train_loss 1.1014 acc 0.113 f1 0.072 || val_loss 1.0963 acc 0.138 f1 0.090
[W14] GRU Epoch 02 | train_loss 1.0964 acc 0.197 f1 0.193 || val_loss 1.0968 acc 0.242 f1 0.234
[W14] GRU Epoch 03 | train_loss 1.0900 acc 0.337 f1 0.331 || val_loss 1.0973 acc 0.283 f1 0.278
[W14] GRU Epoch 04 | train_loss 1.0851 acc 0.375 f1 0.364 || val_loss 1.0978 acc 0.321 f1 0.311
[W14] GRU Epoch 05 | train_loss 1.0791 acc 0.426 f1 0.398 || val_loss 1.0999 acc 0.383 f1 0.360
[W14] GRU Epoch 06 | train_loss 1.0671 acc 0.465 f1 0.433 || val_loss 1.1063 acc 0.379 f1 0.353
[W14] GRU Epoch 07 | train_loss 1.0593 acc 0.445 f1 0.418 || val_loss 1.1159 acc 0.396 f1 0.366
[W14] GRU Epoch 08 | train_loss 1.0444 acc 0.487 f1 0.448 || val_loss 1.1231 acc 0.338 f1 0.327
[W14] GRU Epoch 09 | train_loss 1.0285 acc 0.460 f1 0.433 || val_loss 1.1320 acc 0.358 f1 0.343
[W14] GRU Epoch 10 | train_loss 1.0097 acc 0.495 f1 0.463 || val_loss 1.1368 acc 0.350 f1 0.339
[W14] GRU Epoch 11 | train_loss 0.9901 a

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W14] LSTM Epoch 01 | train_loss 1.1022 acc 0.234 f1 0.197 || val_loss 1.0975 acc 0.263 f1 0.207
[W14] LSTM Epoch 02 | train_loss 1.0974 acc 0.279 f1 0.228 || val_loss 1.0982 acc 0.279 f1 0.215
[W14] LSTM Epoch 03 | train_loss 1.0936 acc 0.364 f1 0.296 || val_loss 1.1019 acc 0.371 f1 0.282
[W14] LSTM Epoch 04 | train_loss 1.0905 acc 0.419 f1 0.373 || val_loss 1.1025 acc 0.346 f1 0.298
[W14] LSTM Epoch 05 | train_loss 1.0865 acc 0.422 f1 0.372 || val_loss 1.1073 acc 0.371 f1 0.301
[W14] LSTM Epoch 06 | train_loss 1.0806 acc 0.451 f1 0.397 || val_loss 1.1160 acc 0.350 f1 0.288
[W14] LSTM Epoch 07 | train_loss 1.0685 acc 0.466 f1 0.415 || val_loss 1.1252 acc 0.371 f1 0.315
[W14] LSTM Epoch 08 | train_loss 1.0560 acc 0.468 f1 0.435 || val_loss 1.1258 acc 0.338 f1 0.320
[W14] LSTM Epoch 09 | train_loss 1.0352 acc 0.440 f1 0.415 || val_loss 1.1587 acc 0.362 f1 0.330
[W14] LSTM Epoch 10 | train_loss 1.0176 acc 0.429 f1 0.408 || val_loss 1.1218 acc 0.279 f1 0.281
[W14] LSTM Epoch 11 | train_lo

Similar to the script above, but instead this script uses SMOTE to augment the imbalanced classes so that the distribution is even.

In [16]:
"""
experiment_runner_smote.py

PyTorch experiment runner with SMOTE augmentation:

- builds sliding windows for window_sizes = [3,4,5,6,7,10,14]
- creates ANN and sequence inputs
- applies SMOTE on training data to balance classes
- trains ANN, CNN1D, RNN, GRU, LSTM
- participant-wise split (train/val/test)
- saves results to CSV, confusion matrix PNGs, and class distributions

Usage:
    python experiment_runner_smote.py
"""

import os
import copy
import random
from pathlib import Path
from collections import Counter
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, precision_recall_fscore_support, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# -------------------------
# Config
# -------------------------
CSV_PATH = "stress_detection.csv"
OUTPUT_DIR = Path("outputs2")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
WINDOW_SIZES = [2,3,4,5,6,7,10,14]
RAW_FEATURES = [
    'Openness','Conscientiousness','Extraversion','Agreeableness','Neuroticism',
    'sleep_time','wake_time','sleep_duration','PSQI_score',
    'call_duration','num_calls','num_sms',
    'screen_on_time','skin_conductance','accelerometer',
    'mobility_radius','mobility_distance'
]
STATIC_PERSONALITY = ['Openness','Conscientiousness','Extraversion','Agreeableness','Neuroticism']
TARGET_COL = 'PSS_score'
CLASS_COL = 'stress_class'

TRAIN_P = 0.7
VAL_P = 0.15
TEST_P = 0.15

EPOCHS = 60
BATCH_SIZE = 64
LR = 1e-3
PATIENCE = 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

MODEL_NAMES = ["ANN","CNN1D","RNN","GRU","LSTM"]

# -------------------------
# Helpers
# -------------------------
def stress_to_class(score):
    s = float(score)
    if s <= 13: return 0
    if s <= 26: return 1
    return 2

def ensure_dir(path):
    Path(path).mkdir(parents=True, exist_ok=True)

# -------------------------
# Window builder
# -------------------------
def build_windows(df, raw_features, window_size=4):
    X_seq, X_flat, y, participants = [], [], [], []

    for pid, df_p in df.groupby("participant_id"):
        df_p = df_p.sort_values("day").reset_index(drop=True)
        n = len(df_p)
        if n <= window_size:
            continue
        arr = df_p[raw_features].values.astype(float)
        for i in range(n - window_size):
            window = arr[i:i+window_size]
            mean_vals = window.mean(axis=0)
            std_vals = window.std(axis=0)
            min_vals = window.min(axis=0)
            max_vals = window.max(axis=0)
            slope_vals = window[-1] - window[0]
            agg = np.concatenate([mean_vals, std_vals, min_vals, max_vals, slope_vals])

            flat = np.concatenate([window.flatten(), agg])
            target_score = df_p[TARGET_COL].iloc[i+window_size]
            cls = stress_to_class(target_score)

            X_seq.append(window.copy())
            X_flat.append(flat.copy())
            y.append(int(cls))
            participants.append(pid)

    return np.array(X_seq), np.array(X_flat), np.array(y), np.array(participants)

# -------------------------
# Dataset
# -------------------------
class StressDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# -------------------------
# Models
# -------------------------
class ANNModel(nn.Module):
    def __init__(self, in_dim, hidden=[256,128,64], num_classes=3, p_drop=0.3):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers.append(nn.Linear(last,h))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(h))
            layers.append(nn.Dropout(p_drop))
            last = h
        layers.append(nn.Linear(last,num_classes))
        self.net = nn.Sequential(*layers)
    def forward(self,x): return self.net(x)

class CNN1DModel(nn.Module):
    def __init__(self, seq_len, feat_dim, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv1d(feat_dim,64,2)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64,128,2)
        self.bn2 = nn.BatchNorm1d(128)
        l = seq_len - (2-1) - (2-1)
        if l<1: l=1
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*1,64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64,num_classes)
        )
    def forward(self,x):
        x = x.permute(0,2,1)
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.gap(x)
        return self.fc(x)

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden=64, num_layers=1, rnn_type='RNN', num_classes=3, dropout=0.2):
        super().__init__()
        if rnn_type=='RNN':
            self.rnn = nn.RNN(input_dim,hidden,num_layers,batch_first=True,nonlinearity='tanh',dropout=dropout)
        elif rnn_type=='GRU':
            self.rnn = nn.GRU(input_dim,hidden,num_layers,batch_first=True,dropout=dropout)
        elif rnn_type=='LSTM':
            self.rnn = nn.LSTM(input_dim,hidden,num_layers,batch_first=True,dropout=dropout)
        else: raise ValueError("Unknown rnn_type")
        self.head = nn.Sequential(nn.Linear(hidden,32), nn.ReLU(), nn.Dropout(0.2), nn.Linear(32,num_classes))
        self.rnn_type = rnn_type
    def forward(self,x):
        out,_ = self.rnn(x)
        return self.head(out[:,-1,:])

# -------------------------
# Training & Evaluation
# -------------------------
def compute_class_weights(y):
    counts = Counter(y.tolist())
    total = sum(counts.values())
    num_classes = len(counts)
    weights = []
    for i in range(max(counts.keys())+1):
        cnt = counts.get(i,0)
        weights.append(total/(num_classes*cnt) if cnt>0 else 0.0)
    return torch.tensor(weights,dtype=torch.float32,device=DEVICE)

def train_one_epoch(model, loader, opt, criterion):
    model.train()
    total_loss = 0
    preds, trues = [], []
    for xb,yb in loader:
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        out = model(xb)
        loss = criterion(out,yb)
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item()*xb.size(0)
        preds.append(out.detach().argmax(1).cpu().numpy())
        trues.append(yb.cpu().numpy())
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    return total_loss/len(trues), accuracy_score(trues,preds), f1_score(trues,preds,average='macro',zero_division=0)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss=0
    preds,trues=[],[]
    with torch.no_grad():
        for xb,yb in loader:
            xb,yb = xb.to(DEVICE), yb.to(DEVICE)
            out = model(xb)
            total_loss += criterion(out,yb).item()*xb.size(0)
            preds.append(out.argmax(1).cpu().numpy())
            trues.append(yb.cpu().numpy())
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mean_loss = total_loss / len(trues)
    acc = accuracy_score(trues,preds)
    f1 = f1_score(trues,preds,average='macro',zero_division=0)
    prec, rec, f1s, sup = precision_recall_fscore_support(trues,preds,average=None,zero_division=0)
    return mean_loss, acc, f1, prec, rec, f1s, sup, preds, trues

# -------------------------
# Experiment for a window
# -------------------------
def run_experiment_for_window(df, window_size, model_name, train_pids, val_pids, test_pids):
    X_seq, X_flat, y, pids = build_windows(df, RAW_FEATURES, window_size)
    if len(y)==0: return None

    # split by participant
    train_mask = np.isin(pids, train_pids)
    val_mask = np.isin(pids, val_pids)
    test_mask = np.isin(pids, test_pids)

    X_seq_train, X_seq_val, X_seq_test = X_seq[train_mask], X_seq[val_mask], X_seq[test_mask]
    X_flat_train, X_flat_val, X_flat_test = X_flat[train_mask], X_flat[val_mask], X_flat[test_mask]
    y_train, y_val, y_test = y[train_mask], y[val_mask], y[test_mask]

    if len(y_train)==0 or len(y_val)==0 or len(y_test)==0:
        print("One split empty for window", window_size)
        return None

    # -------------------------
    # SMOTE augmentation (training set only)
    # -------------------------
    print(f"Class distribution BEFORE SMOTE: {Counter(y_train)}")
    smote = SMOTE(random_state=SEED)
    if model_name=="ANN":
        X_train_aug, y_train_aug = smote.fit_resample(X_flat_train, y_train)
        X_train_aug = X_train_aug
    else:
        N,L,F = X_seq_train.shape
        X_seq_flat = X_seq_train.reshape(N,L*F)
        X_train_aug, y_train_aug = smote.fit_resample(X_seq_flat, y_train)
        X_train_aug = X_train_aug.reshape(-1,L,F)
    print(f"Class distribution AFTER SMOTE: {Counter(y_train_aug)}")

    # save class distributions
    dist_df = pd.DataFrame({'Class':[0,1,2],
                            'Before': [Counter(y_train)[0],Counter(y_train)[1],Counter(y_train)[2]],
                            'After': [Counter(y_train_aug)[0],Counter(y_train_aug)[1],Counter(y_train_aug)[2]]})
    dist_df.to_csv(OUTPUT_DIR/f'class_distribution_w{window_size}_{model_name}.csv', index=False)

    # -------------------------
    # Scaling
    # -------------------------
    if model_name=="ANN":
        scaler = StandardScaler().fit(X_train_aug)
        Xtr = scaler.transform(X_train_aug)
        Xv = scaler.transform(X_flat_val)
        Xt = scaler.transform(X_flat_test)
        input_dim = Xtr.shape[1]
    else:
        N,L,F = X_train_aug.shape
        scaler = StandardScaler().fit(X_train_aug.reshape(-1,F))
        Xtr = scaler.transform(X_train_aug.reshape(-1,F)).reshape(N,L,F)
        Xv = scaler.transform(X_seq_val.reshape(-1,F)).reshape(X_seq_val.shape)
        Xt = scaler.transform(X_seq_test.reshape(-1,F)).reshape(X_seq_test.shape)
        input_dim = F

    train_ds = StressDataset(Xtr, y_train_aug)
    val_ds = StressDataset(Xv, y_val)
    test_ds = StressDataset(Xt, y_test)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    # -------------------------
    # Model
    # -------------------------
    if model_name=="ANN": model = ANNModel(input_dim).to(DEVICE)
    elif model_name=="CNN1D": model = CNN1DModel(Xtr.shape[1], Xtr.shape[2]).to(DEVICE)
    elif model_name in ("RNN","GRU","LSTM"): model = RNNModel(input_dim, hidden=64, num_layers=1, rnn_type=model_name).to(DEVICE)
    else: raise ValueError("Unknown model")

    # loss and optimizer
    class_weights = compute_class_weights(torch.tensor(y_train_aug))
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)

    # -------------------------
    # Training loop with early stopping
    # -------------------------
    best_val_f1, best_state, cur_wait = -1.0, None, 0
    for epoch in range(1,EPOCHS+1):
        train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, opt, criterion)
        val_loss, val_acc, val_f1, *_ = evaluate(model, val_loader, criterion)
        print(f"[W{window_size}] {model_name} Epoch {epoch:02d} | train_loss {train_loss:.4f} acc {train_acc:.3f} f1 {train_f1:.3f} || val_loss {val_loss:.4f} acc {val_acc:.3f} f1 {val_f1:.3f}")
        if val_f1 > best_val_f1+1e-4:
            best_val_f1 = val_f1
            best_state = copy.deepcopy(model.state_dict())
            cur_wait = 0
        else:
            cur_wait += 1
            if cur_wait >= PATIENCE:
                print("Early stopping.")
                break

    # test
    if best_state is not None: model.load_state_dict(best_state)
    test_loss, test_acc, test_f1, prec, rec, f1s, sup, preds, trues = evaluate(model, test_loader, criterion)

    # save confusion matrix
    cm = confusion_matrix(trues,preds)
    np.save(OUTPUT_DIR/f"cm_w{window_size}_{model_name}.npy", cm)
    plt.figure(figsize=(4,4))
    plt.imshow(cm, cmap='Blues')
    plt.title(f"CM W{window_size} {model_name}")
    plt.colorbar()
    plt.xlabel("pred")
    plt.ylabel("true")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR/f"cm_w{window_size}_{model_name}.png")
    plt.close()

    return {
        "window": window_size,
        "model": model_name,
        "train_samples": len(y_train_aug),
        "val_samples": len(y_val),
        "test_samples": len(y_test),
        "test_loss": float(test_loss),
        "test_acc": float(test_acc),
        "test_macro_f1": float(test_f1),
        "per_class_prec": prec.tolist(),
        "per_class_rec": rec.tolist(),
        "per_class_f1": f1s.tolist(),
        "support": sup.tolist()
    }

# -------------------------
# Main
# -------------------------
def main():
    print("Device:", DEVICE)
    df = pd.read_csv(CSV_PATH)
    df = df.sort_values(['participant_id','day']).reset_index(drop=True)
    df[CLASS_COL] = df[TARGET_COL].apply(stress_to_class)

    # participant split
    pids = df['participant_id'].unique()
    random.shuffle(pids)
    n = len(pids)
    n_train = int(n*TRAIN_P)
    n_val = int(n*VAL_P)
    train_pids = pids[:n_train]
    val_pids = pids[n_train:n_train+n_val]
    test_pids = pids[n_train+n_val:]
    print(f"Participants: total {n} train {len(train_pids)} val {len(val_pids)} test {len(test_pids)}")

    results=[]
    for w in WINDOW_SIZES:
        for mname in MODEL_NAMES:
            print("="*60)
            print(f"Running Window={w} Model={mname}")
            try:
                res = run_experiment_for_window(df, w, mname, train_pids, val_pids, test_pids)
                if res is not None:
                    results.append(res)
                    pd.DataFrame(results).to_csv(OUTPUT_DIR/"results_summary.csv", index=False)
            except Exception as e:
                print("Error running", w, mname, e)

    if results:
        pd.DataFrame(results).to_csv(OUTPUT_DIR/"results_summary.csv", index=False)
        print("Saved results to", OUTPUT_DIR/"results_summary.csv")
    else:
        print("No results to save.")

if __name__=="__main__":
    main()


Device: cpu
Participants: total 100 train 70 val 15 test 15
Running Window=2 Model=ANN
Class distribution BEFORE SMOTE: Counter({np.int64(2): 879, np.int64(1): 846, np.int64(0): 235})
Class distribution AFTER SMOTE: Counter({np.int64(1): 879, np.int64(0): 879, np.int64(2): 879})
[W2] ANN Epoch 01 | train_loss 1.1238 acc 0.398 f1 0.396 || val_loss 1.1818 acc 0.274 f1 0.266
[W2] ANN Epoch 02 | train_loss 0.9576 acc 0.514 f1 0.505 || val_loss 1.1786 acc 0.310 f1 0.296
[W2] ANN Epoch 03 | train_loss 0.8963 acc 0.553 f1 0.543 || val_loss 1.1767 acc 0.350 f1 0.320
[W2] ANN Epoch 04 | train_loss 0.8247 acc 0.601 f1 0.594 || val_loss 1.1620 acc 0.350 f1 0.314
[W2] ANN Epoch 05 | train_loss 0.7570 acc 0.637 f1 0.632 || val_loss 1.1716 acc 0.379 f1 0.331
[W2] ANN Epoch 06 | train_loss 0.7207 acc 0.650 f1 0.645 || val_loss 1.2087 acc 0.379 f1 0.319
[W2] ANN Epoch 07 | train_loss 0.6841 acc 0.674 f1 0.670 || val_loss 1.2106 acc 0.388 f1 0.337
[W2] ANN Epoch 08 | train_loss 0.6473 acc 0.688 f1 0.68

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W2] RNN Epoch 01 | train_loss 1.0954 acc 0.353 f1 0.339 || val_loss 1.1019 acc 0.321 f1 0.304
[W2] RNN Epoch 02 | train_loss 1.0761 acc 0.421 f1 0.416 || val_loss 1.1055 acc 0.321 f1 0.305
[W2] RNN Epoch 03 | train_loss 1.0559 acc 0.451 f1 0.446 || val_loss 1.1059 acc 0.324 f1 0.307
[W2] RNN Epoch 04 | train_loss 1.0421 acc 0.455 f1 0.449 || val_loss 1.1171 acc 0.329 f1 0.313
[W2] RNN Epoch 05 | train_loss 1.0306 acc 0.462 f1 0.453 || val_loss 1.1213 acc 0.333 f1 0.311
[W2] RNN Epoch 06 | train_loss 1.0138 acc 0.487 f1 0.481 || val_loss 1.1239 acc 0.324 f1 0.305
[W2] RNN Epoch 07 | train_loss 0.9993 acc 0.495 f1 0.488 || val_loss 1.1134 acc 0.362 f1 0.328
[W2] RNN Epoch 08 | train_loss 0.9857 acc 0.508 f1 0.499 || val_loss 1.1212 acc 0.331 f1 0.304
[W2] RNN Epoch 09 | train_loss 0.9590 acc 0.528 f1 0.521 || val_loss 1.1228 acc 0.340 f1 0.312
[W2] RNN Epoch 10 | train_loss 0.9449 acc 0.536 f1 0.528 || val_loss 1.1179 acc 0.355 f1 0.326
[W2] RNN Epoch 11 | train_loss 0.9328 acc 0.555 f1

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W2] GRU Epoch 02 | train_loss 1.0864 acc 0.421 f1 0.409 || val_loss 1.0872 acc 0.374 f1 0.342
[W2] GRU Epoch 03 | train_loss 1.0707 acc 0.453 f1 0.446 || val_loss 1.0767 acc 0.395 f1 0.365
[W2] GRU Epoch 04 | train_loss 1.0442 acc 0.479 f1 0.468 || val_loss 1.0805 acc 0.362 f1 0.335
[W2] GRU Epoch 05 | train_loss 1.0102 acc 0.491 f1 0.478 || val_loss 1.1135 acc 0.336 f1 0.318
[W2] GRU Epoch 06 | train_loss 0.9756 acc 0.522 f1 0.504 || val_loss 1.1266 acc 0.360 f1 0.336
[W2] GRU Epoch 07 | train_loss 0.9385 acc 0.545 f1 0.531 || val_loss 1.1253 acc 0.371 f1 0.337
[W2] GRU Epoch 08 | train_loss 0.8960 acc 0.567 f1 0.557 || val_loss 1.1606 acc 0.357 f1 0.330
[W2] GRU Epoch 09 | train_loss 0.8511 acc 0.586 f1 0.574 || val_loss 1.1568 acc 0.417 f1 0.373
[W2] GRU Epoch 10 | train_loss 0.8186 acc 0.614 f1 0.606 || val_loss 1.1796 acc 0.386 f1 0.348
[W2] GRU Epoch 11 | train_loss 0.7826 acc 0.623 f1 0.615 || val_loss 1.1929 acc 0.412 f1 0.355
[W2] GRU Epoch 12 | train_loss 0.7476 acc 0.640 f1

...

[W14] RNN Epoch 02 | train_loss 1.0782 acc 0.442 f1 0.367 || val_loss 1.1081 acc 0.296 f1 0.235
[W14] RNN Epoch 03 | train_loss 1.0549 acc 0.465 f1 0.404 || val_loss 1.1097 acc 0.292 f1 0.249
[W14] RNN Epoch 04 | train_loss 1.0132 acc 0.509 f1 0.463 || val_loss 1.1248 acc 0.312 f1 0.273
[W14] RNN Epoch 05 | train_loss 0.9623 acc 0.521 f1 0.495 || val_loss 1.1258 acc 0.317 f1 0.274
[W14] RNN Epoch 06 | train_loss 0.9225 acc 0.547 f1 0.531 || val_loss 1.1682 acc 0.317 f1 0.291
[W14] RNN Epoch 07 | train_loss 0.8755 acc 0.568 f1 0.554 || val_loss 1.1908 acc 0.367 f1 0.333
[W14] RNN Epoch 08 | train_loss 0.8272 acc 0.599 f1 0.584 || val_loss 1.2176 acc 0.383 f1 0.349
[W14] RNN Epoch 09 | train_loss 0.7924 acc 0.630 f1 0.616 || val_loss 1.2539 acc 0.388 f1 0.358
[W14] RNN Epoch 10 | train_loss 0.7521 acc 0.632 f1 0.618 || val_loss 1.2892 acc 0.375 f1 0.327
[W14] RNN Epoch 11 | train_loss 0.7281 acc 0.649 f1 0.636 || val_loss 1.3134 acc 0.371 f1 0.331
[W14] RNN Epoch 12 | train_loss 0.6830 a

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W14] GRU Epoch 01 | train_loss 1.1018 acc 0.325 f1 0.236 || val_loss 1.0952 acc 0.408 f1 0.236
[W14] GRU Epoch 02 | train_loss 1.0909 acc 0.388 f1 0.341 || val_loss 1.0969 acc 0.400 f1 0.333
[W14] GRU Epoch 03 | train_loss 1.0830 acc 0.429 f1 0.425 || val_loss 1.0974 acc 0.383 f1 0.336
[W14] GRU Epoch 04 | train_loss 1.0694 acc 0.468 f1 0.470 || val_loss 1.0959 acc 0.383 f1 0.360
[W14] GRU Epoch 05 | train_loss 1.0470 acc 0.492 f1 0.492 || val_loss 1.1017 acc 0.342 f1 0.326
[W14] GRU Epoch 06 | train_loss 1.0022 acc 0.514 f1 0.509 || val_loss 1.1532 acc 0.317 f1 0.312
[W14] GRU Epoch 07 | train_loss 0.9185 acc 0.566 f1 0.557 || val_loss 1.1322 acc 0.350 f1 0.315
[W14] GRU Epoch 08 | train_loss 0.8012 acc 0.601 f1 0.602 || val_loss 1.1600 acc 0.392 f1 0.343
[W14] GRU Epoch 09 | train_loss 0.7615 acc 0.627 f1 0.629 || val_loss 1.1760 acc 0.375 f1 0.340
[W14] GRU Epoch 10 | train_loss 0.7219 acc 0.635 f1 0.636 || val_loss 1.0937 acc 0.442 f1 0.348
[W14] GRU Epoch 11 | train_loss 0.7217 a

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W14] LSTM Epoch 01 | train_loss 1.1082 acc 0.331 f1 0.167 || val_loss 1.1157 acc 0.442 f1 0.204
[W14] LSTM Epoch 02 | train_loss 1.0992 acc 0.374 f1 0.294 || val_loss 1.1110 acc 0.267 f1 0.218
[W14] LSTM Epoch 03 | train_loss 1.0899 acc 0.424 f1 0.376 || val_loss 1.1060 acc 0.300 f1 0.297
[W14] LSTM Epoch 04 | train_loss 1.0656 acc 0.450 f1 0.427 || val_loss 1.0932 acc 0.350 f1 0.333
[W14] LSTM Epoch 05 | train_loss 0.9971 acc 0.484 f1 0.468 || val_loss 1.1478 acc 0.312 f1 0.307
[W14] LSTM Epoch 06 | train_loss 0.8704 acc 0.543 f1 0.536 || val_loss 1.0666 acc 0.408 f1 0.325
[W14] LSTM Epoch 07 | train_loss 0.8146 acc 0.583 f1 0.577 || val_loss 1.1165 acc 0.400 f1 0.338
[W14] LSTM Epoch 08 | train_loss 0.7683 acc 0.618 f1 0.618 || val_loss 1.3950 acc 0.362 f1 0.326
[W14] LSTM Epoch 09 | train_loss 0.7588 acc 0.614 f1 0.615 || val_loss 1.3070 acc 0.342 f1 0.311
[W14] LSTM Epoch 10 | train_loss 0.7435 acc 0.609 f1 0.607 || val_loss 1.2141 acc 0.433 f1 0.364
[W14] LSTM Epoch 11 | train_lo

In [ ]:
"""
experiment_runner_smote.py

PyTorch experiment runner with SMOTE augmentation:

- builds sliding windows for window_sizes = [2,3,4,5,6,7,10,14]
- creates ANN and sequence inputs
- applies SMOTE on training data to balance classes
- trains ANN, CNN1D, RNN, GRU, LSTM
- participant-wise split (train/val/test)
- saves results to CSV, confusion matrix PNGs, and class distributions

Usage:
    python experiment_runner_smote.py
"""

import os
import copy
import random
from pathlib import Path
from collections import Counter
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, precision_recall_fscore_support, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# -------------------------
# Config
# -------------------------
CSV_PATH = "stress_detection.csv"
OUTPUT_DIR = Path("outputs3")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
WINDOW_SIZES = [2,3,4,5,6,7,10,14]
RAW_FEATURES = [
    'Openness','Conscientiousness','Extraversion','Agreeableness','Neuroticism',
    'sleep_time','wake_time','sleep_duration','PSQI_score',
    'call_duration','num_calls','num_sms',
    'screen_on_time','skin_conductance','accelerometer',
    'mobility_radius','mobility_distance'
]
STATIC_PERSONALITY = ['Openness','Conscientiousness','Extraversion','Agreeableness','Neuroticism']
TARGET_COL = 'PSS_score'
CLASS_COL = 'stress_class'

TRAIN_P = 0.7
VAL_P = 0.15
TEST_P = 0.15

EPOCHS = 60
BATCH_SIZE = 64
LR = 1e-3
PATIENCE = 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

MODEL_NAMES = ["ANN","CNN1D","RNN","GRU","LSTM"]

# -------------------------
# Helpers
# -------------------------
def stress_to_class(score):
    """
    Equal-width split of 10-40 into 3 classes.
    Class 0: 10-19.99
    Class 1: 20-29.99
    Class 2: 30-40
    """
    s = float(score)
    if s < 20: return 0
    elif s < 30: return 1
    else: return 2

def ensure_dir(path):
    Path(path).mkdir(parents=True, exist_ok=True)

# -------------------------
# Window builder
# -------------------------
def build_windows(df, raw_features, window_size=4):
    X_seq, X_flat, y, participants = [], [], [], []

    for pid, df_p in df.groupby("participant_id"):
        df_p = df_p.sort_values("day").reset_index(drop=True)
        n = len(df_p)
        if n <= window_size:
            continue
        arr = df_p[raw_features].values.astype(float)
        for i in range(n - window_size):
            window = arr[i:i+window_size]
            mean_vals = window.mean(axis=0)
            std_vals = window.std(axis=0)
            min_vals = window.min(axis=0)
            max_vals = window.max(axis=0)
            slope_vals = window[-1] - window[0]
            agg = np.concatenate([mean_vals, std_vals, min_vals, max_vals, slope_vals])

            flat = np.concatenate([window.flatten(), agg])
            target_score = df_p[TARGET_COL].iloc[i+window_size]
            cls = stress_to_class(target_score)

            X_seq.append(window.copy())
            X_flat.append(flat.copy())
            y.append(int(cls))
            participants.append(pid)

    return np.array(X_seq), np.array(X_flat), np.array(y), np.array(participants)

# -------------------------
# Dataset
# -------------------------
class StressDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# -------------------------
# Models
# -------------------------
class ANNModel(nn.Module):
    def __init__(self, in_dim, hidden=[256,128,64], num_classes=3, p_drop=0.3):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers.append(nn.Linear(last,h))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(h))
            layers.append(nn.Dropout(p_drop))
            last = h
        layers.append(nn.Linear(last,num_classes))
        self.net = nn.Sequential(*layers)
    def forward(self,x): return self.net(x)

class CNN1DModel(nn.Module):
    def __init__(self, seq_len, feat_dim, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv1d(feat_dim,64,2)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64,128,2)
        self.bn2 = nn.BatchNorm1d(128)
        l = seq_len - (2-1) - (2-1)
        if l<1: l=1
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*1,64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64,num_classes)
        )
    def forward(self,x):
        x = x.permute(0,2,1)
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.gap(x)
        return self.fc(x)

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden=64, num_layers=1, rnn_type='RNN', num_classes=3, dropout=0.2):
        super().__init__()
        if rnn_type=='RNN':
            self.rnn = nn.RNN(input_dim,hidden,num_layers,batch_first=True,nonlinearity='tanh',dropout=dropout)
        elif rnn_type=='GRU':
            self.rnn = nn.GRU(input_dim,hidden,num_layers,batch_first=True,dropout=dropout)
        elif rnn_type=='LSTM':
            self.rnn = nn.LSTM(input_dim,hidden,num_layers,batch_first=True,dropout=dropout)
        else: raise ValueError("Unknown rnn_type")
        self.head = nn.Sequential(nn.Linear(hidden,32), nn.ReLU(), nn.Dropout(0.2), nn.Linear(32,num_classes))
        self.rnn_type = rnn_type
    def forward(self,x):
        out,_ = self.rnn(x)
        return self.head(out[:,-1,:])

# -------------------------
# Training & Evaluation
# -------------------------
def compute_class_weights(y):
    counts = Counter(y.tolist())
    total = sum(counts.values())
    num_classes = len(counts)
    weights = []
    for i in range(max(counts.keys())+1):
        cnt = counts.get(i,0)
        weights.append(total/(num_classes*cnt) if cnt>0 else 0.0)
    return torch.tensor(weights,dtype=torch.float32,device=DEVICE)

def train_one_epoch(model, loader, opt, criterion):
    model.train()
    total_loss = 0
    preds, trues = [], []
    for xb,yb in loader:
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        out = model(xb)
        loss = criterion(out,yb)
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item()*xb.size(0)
        preds.append(out.detach().argmax(1).cpu().numpy())
        trues.append(yb.cpu().numpy())
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    return total_loss/len(trues), accuracy_score(trues,preds), f1_score(trues,preds,average='macro',zero_division=0)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss=0
    preds,trues=[],[]
    with torch.no_grad():
        for xb,yb in loader:
            xb,yb = xb.to(DEVICE), yb.to(DEVICE)
            out = model(xb)
            total_loss += criterion(out,yb).item()*xb.size(0)
            preds.append(out.argmax(1).cpu().numpy())
            trues.append(yb.cpu().numpy())
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mean_loss = total_loss / len(trues)
    acc = accuracy_score(trues,preds)
    f1 = f1_score(trues,preds,average='macro',zero_division=0)
    prec, rec, f1s, sup = precision_recall_fscore_support(trues,preds,average=None,zero_division=0)
    return mean_loss, acc, f1, prec, rec, f1s, sup, preds, trues

# -------------------------
# Experiment for a window
# -------------------------
def run_experiment_for_window(df, window_size, model_name, train_pids, val_pids, test_pids):
    X_seq, X_flat, y, pids = build_windows(df, RAW_FEATURES, window_size)
    if len(y)==0: return None

    # split by participant
    train_mask = np.isin(pids, train_pids)
    val_mask = np.isin(pids, val_pids)
    test_mask = np.isin(pids, test_pids)

    X_seq_train, X_seq_val, X_seq_test = X_seq[train_mask], X_seq[val_mask], X_seq[test_mask]
    X_flat_train, X_flat_val, X_flat_test = X_flat[train_mask], X_flat[val_mask], X_flat[test_mask]
    y_train, y_val, y_test = y[train_mask], y[val_mask], y[test_mask]

    if len(y_train)==0 or len(y_val)==0 or len(y_test)==0:
        print("One split empty for window", window_size)
        return None

    # -------------------------
    # SMOTE augmentation (training set only)
    # -------------------------
    print(f"Class distribution BEFORE SMOTE: {Counter(y_train)}")
    smote = SMOTE(random_state=SEED)
    if model_name=="ANN":
        X_train_aug, y_train_aug = smote.fit_resample(X_flat_train, y_train)
        X_train_aug = X_train_aug
    else:
        N,L,F = X_seq_train.shape
        X_seq_flat = X_seq_train.reshape(N,L*F)
        X_train_aug, y_train_aug = smote.fit_resample(X_seq_flat, y_train)
        X_train_aug = X_train_aug.reshape(-1,L,F)
    print(f"Class distribution AFTER SMOTE: {Counter(y_train_aug)}")

    # save class distributions
    dist_df = pd.DataFrame({'Class':[0,1,2],
                            'Before': [Counter(y_train)[0],Counter(y_train)[1],Counter(y_train)[2]],
                            'After': [Counter(y_train_aug)[0],Counter(y_train_aug)[1],Counter(y_train_aug)[2]]})
    dist_df.to_csv(OUTPUT_DIR/f'class_distribution_w{window_size}_{model_name}.csv', index=False)

    # -------------------------
    # Scaling
    # -------------------------
    if model_name=="ANN":
        scaler = StandardScaler().fit(X_train_aug)
        Xtr = scaler.transform(X_train_aug)
        Xv = scaler.transform(X_flat_val)
        Xt = scaler.transform(X_flat_test)
        input_dim = Xtr.shape[1]
    else:
        N,L,F = X_train_aug.shape
        scaler = StandardScaler().fit(X_train_aug.reshape(-1,F))
        Xtr = scaler.transform(X_train_aug.reshape(-1,F)).reshape(N,L,F)
        Xv = scaler.transform(X_seq_val.reshape(-1,F)).reshape(X_seq_val.shape)
        Xt = scaler.transform(X_seq_test.reshape(-1,F)).reshape(X_seq_test.shape)
        input_dim = F

    train_ds = StressDataset(Xtr, y_train_aug)
    val_ds = StressDataset(Xv, y_val)
    test_ds = StressDataset(Xt, y_test)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    # -------------------------
    # Model
    # -------------------------
    if model_name=="ANN": model = ANNModel(input_dim).to(DEVICE)
    elif model_name=="CNN1D": model = CNN1DModel(Xtr.shape[1], Xtr.shape[2]).to(DEVICE)
    elif model_name in ("RNN","GRU","LSTM"): model = RNNModel(input_dim, hidden=64, num_layers=1, rnn_type=model_name).to(DEVICE)
    else: raise ValueError("Unknown model")

    # loss and optimizer
    class_weights = compute_class_weights(torch.tensor(y_train_aug))
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)

    # -------------------------
    # Training loop with early stopping
    # -------------------------
    best_val_f1, best_state, cur_wait = -1.0, None, 0
    for epoch in range(1,EPOCHS+1):
        train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, opt, criterion)
        val_loss, val_acc, val_f1, *_ = evaluate(model, val_loader, criterion)
        print(f"[W{window_size}] {model_name} Epoch {epoch:02d} | train_loss {train_loss:.4f} acc {train_acc:.3f} f1 {train_f1:.3f} || val_loss {val_loss:.4f} acc {val_acc:.3f} f1 {val_f1:.3f}")
        if val_f1 > best_val_f1+1e-4:
            best_val_f1 = val_f1
            best_state = copy.deepcopy(model.state_dict())
            cur_wait = 0
        else:
            cur_wait += 1
            if cur_wait >= PATIENCE:
                print("Early stopping.")
                break

    # test
    if best_state is not None: model.load_state_dict(best_state)
    test_loss, test_acc, test_f1, prec, rec, f1s, sup, preds, trues = evaluate(model, test_loader, criterion)

    # save confusion matrix
    cm = confusion_matrix(trues,preds)
    np.save(OUTPUT_DIR/f"cm_w{window_size}_{model_name}.npy", cm)
    plt.figure(figsize=(4,4))
    plt.imshow(cm, cmap='Blues')
    plt.title(f"CM W{window_size} {model_name}")
    plt.colorbar()
    plt.xlabel("pred")
    plt.ylabel("true")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR/f"cm_w{window_size}_{model_name}.png")
    plt.close()

    return {
        "window": window_size,
        "model": model_name,
        "train_samples": len(y_train_aug),
        "val_samples": len(y_val),
        "test_samples": len(y_test),
        "test_loss": float(test_loss),
        "test_acc": float(test_acc),
        "test_macro_f1": float(test_f1),
        "per_class_prec": prec.tolist(),
        "per_class_rec": rec.tolist(),
        "per_class_f1": f1s.tolist(),
        "support": sup.tolist()
    }

# -------------------------
# Main
# -------------------------
def main():
    print("Device:", DEVICE)
    df = pd.read_csv(CSV_PATH)
    df = df.sort_values(['participant_id','day']).reset_index(drop=True)
    df[CLASS_COL] = df[TARGET_COL].apply(stress_to_class)

    # participant split
    pids = df['participant_id'].unique()
    random.shuffle(pids)
    n = len(pids)
    n_train = int(n*TRAIN_P)
    n_val = int(n*VAL_P)
    train_pids = pids[:n_train]
    val_pids = pids[n_train:n_train+n_val]
    test_pids = pids[n_train+n_val:]
    print(f"Participants: total {n} train {len(train_pids)} val {len(val_pids)} test {len(test_pids)}")

    results=[]
    for w in WINDOW_SIZES:
        for mname in MODEL_NAMES:
            print("="*60)
            print(f"Running Window={w} Model={mname}")
            try:
                res = run_experiment_for_window(df, w, mname, train_pids, val_pids, test_pids)
                if res is not None:
                    results.append(res)
                    pd.DataFrame(results).to_csv(OUTPUT_DIR/"results_summary.csv", index=False)
            except Exception as e:
                print("Error running", w, mname, e)

    if results:
        pd.DataFrame(results).to_csv(OUTPUT_DIR/"results_summary.csv", index=False)
        print("Saved results to", OUTPUT_DIR/"results_summary.csv")
    else:
        print("No results to save.")

if __name__=="__main__":
    main()


Device: cpu
Participants: total 100 train 70 val 15 test 15
Running Window=2 Model=ANN
Class distribution BEFORE SMOTE: Counter({np.int64(2): 679, np.int64(1): 655, np.int64(0): 626})
Class distribution AFTER SMOTE: Counter({np.int64(1): 679, np.int64(0): 679, np.int64(2): 679})
[W2] ANN Epoch 01 | train_loss 1.2072 acc 0.327 f1 0.326 || val_loss 1.1125 acc 0.352 f1 0.351
[W2] ANN Epoch 02 | train_loss 1.1158 acc 0.382 f1 0.381 || val_loss 1.1183 acc 0.357 f1 0.356
[W2] ANN Epoch 03 | train_loss 1.0770 acc 0.419 f1 0.419 || val_loss 1.1218 acc 0.300 f1 0.300
[W2] ANN Epoch 04 | train_loss 1.0358 acc 0.471 f1 0.471 || val_loss 1.1314 acc 0.340 f1 0.341
[W2] ANN Epoch 05 | train_loss 0.9950 acc 0.503 f1 0.503 || val_loss 1.1502 acc 0.324 f1 0.324
[W2] ANN Epoch 06 | train_loss 0.9727 acc 0.524 f1 0.524 || val_loss 1.1742 acc 0.307 f1 0.306
[W2] ANN Epoch 07 | train_loss 0.9598 acc 0.545 f1 0.545 || val_loss 1.1912 acc 0.307 f1 0.306
[W2] ANN Epoch 08 | train_loss 0.9253 acc 0.554 f1 0.55

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W2] RNN Epoch 02 | train_loss 1.0914 acc 0.374 f1 0.363 || val_loss 1.1040 acc 0.336 f1 0.330
[W2] RNN Epoch 03 | train_loss 1.0840 acc 0.405 f1 0.403 || val_loss 1.1059 acc 0.310 f1 0.309
[W2] RNN Epoch 04 | train_loss 1.0788 acc 0.418 f1 0.418 || val_loss 1.1078 acc 0.293 f1 0.291
[W2] RNN Epoch 05 | train_loss 1.0715 acc 0.424 f1 0.423 || val_loss 1.1136 acc 0.307 f1 0.305
[W2] RNN Epoch 06 | train_loss 1.0689 acc 0.426 f1 0.425 || val_loss 1.1152 acc 0.312 f1 0.311
[W2] RNN Epoch 07 | train_loss 1.0619 acc 0.436 f1 0.436 || val_loss 1.1179 acc 0.314 f1 0.313
[W2] RNN Epoch 08 | train_loss 1.0554 acc 0.449 f1 0.449 || val_loss 1.1211 acc 0.314 f1 0.309
[W2] RNN Epoch 09 | train_loss 1.0507 acc 0.452 f1 0.451 || val_loss 1.1249 acc 0.321 f1 0.321
[W2] RNN Epoch 10 | train_loss 1.0409 acc 0.462 f1 0.462 || val_loss 1.1241 acc 0.310 f1 0.307
Early stopping.
Running Window=2 Model=GRU
Class distribution BEFORE SMOTE: Counter({np.int64(2): 679, np.int64(1): 655, np.int64(0): 626})
Class

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W2] GRU Epoch 02 | train_loss 1.0965 acc 0.366 f1 0.349 || val_loss 1.0994 acc 0.333 f1 0.324
[W2] GRU Epoch 03 | train_loss 1.0929 acc 0.375 f1 0.373 || val_loss 1.0997 acc 0.314 f1 0.315
[W2] GRU Epoch 04 | train_loss 1.0893 acc 0.399 f1 0.399 || val_loss 1.1008 acc 0.319 f1 0.320
[W2] GRU Epoch 05 | train_loss 1.0855 acc 0.405 f1 0.404 || val_loss 1.1009 acc 0.326 f1 0.326
[W2] GRU Epoch 06 | train_loss 1.0785 acc 0.424 f1 0.424 || val_loss 1.1028 acc 0.329 f1 0.329
[W2] GRU Epoch 07 | train_loss 1.0746 acc 0.425 f1 0.425 || val_loss 1.1054 acc 0.319 f1 0.319
[W2] GRU Epoch 08 | train_loss 1.0701 acc 0.425 f1 0.424 || val_loss 1.1049 acc 0.321 f1 0.322
[W2] GRU Epoch 09 | train_loss 1.0628 acc 0.423 f1 0.423 || val_loss 1.1089 acc 0.336 f1 0.337
[W2] GRU Epoch 10 | train_loss 1.0545 acc 0.449 f1 0.448 || val_loss 1.1125 acc 0.338 f1 0.338
[W2] GRU Epoch 11 | train_loss 1.0465 acc 0.459 f1 0.458 || val_loss 1.1156 acc 0.350 f1 0.350
[W2] GRU Epoch 12 | train_loss 1.0330 acc 0.468 f1

...

[W14] RNN Epoch 02 | train_loss 1.0872 acc 0.407 f1 0.393 || val_loss 1.1050 acc 0.338 f1 0.327
[W14] RNN Epoch 03 | train_loss 1.0794 acc 0.429 f1 0.420 || val_loss 1.1107 acc 0.333 f1 0.329
[W14] RNN Epoch 04 | train_loss 1.0675 acc 0.451 f1 0.450 || val_loss 1.1182 acc 0.300 f1 0.300
[W14] RNN Epoch 05 | train_loss 1.0573 acc 0.465 f1 0.465 || val_loss 1.1287 acc 0.317 f1 0.317
[W14] RNN Epoch 06 | train_loss 1.0455 acc 0.466 f1 0.462 || val_loss 1.1436 acc 0.312 f1 0.309
[W14] RNN Epoch 07 | train_loss 1.0319 acc 0.475 f1 0.473 || val_loss 1.1541 acc 0.300 f1 0.300
[W14] RNN Epoch 08 | train_loss 1.0211 acc 0.487 f1 0.481 || val_loss 1.1632 acc 0.304 f1 0.303
[W14] RNN Epoch 09 | train_loss 1.0056 acc 0.511 f1 0.508 || val_loss 1.1732 acc 0.304 f1 0.301
[W14] RNN Epoch 10 | train_loss 0.9996 acc 0.524 f1 0.521 || val_loss 1.1723 acc 0.321 f1 0.321
[W14] RNN Epoch 11 | train_loss 0.9809 acc 0.519 f1 0.514 || val_loss 1.1901 acc 0.317 f1 0.316
Early stopping.
Running Window=14 Model=

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W14] GRU Epoch 01 | train_loss 1.0971 acc 0.347 f1 0.288 || val_loss 1.1005 acc 0.354 f1 0.307
[W14] GRU Epoch 02 | train_loss 1.0920 acc 0.389 f1 0.377 || val_loss 1.1022 acc 0.321 f1 0.308
[W14] GRU Epoch 03 | train_loss 1.0875 acc 0.418 f1 0.415 || val_loss 1.1031 acc 0.317 f1 0.312
[W14] GRU Epoch 04 | train_loss 1.0818 acc 0.416 f1 0.407 || val_loss 1.1059 acc 0.342 f1 0.327
[W14] GRU Epoch 05 | train_loss 1.0732 acc 0.434 f1 0.423 || val_loss 1.1120 acc 0.317 f1 0.305
[W14] GRU Epoch 06 | train_loss 1.0631 acc 0.464 f1 0.459 || val_loss 1.1230 acc 0.317 f1 0.314
[W14] GRU Epoch 07 | train_loss 1.0541 acc 0.463 f1 0.462 || val_loss 1.1387 acc 0.317 f1 0.316
[W14] GRU Epoch 08 | train_loss 1.0498 acc 0.469 f1 0.468 || val_loss 1.1404 acc 0.312 f1 0.313
[W14] GRU Epoch 09 | train_loss 1.0348 acc 0.477 f1 0.475 || val_loss 1.1501 acc 0.308 f1 0.307
[W14] GRU Epoch 10 | train_loss 1.0231 acc 0.486 f1 0.486 || val_loss 1.1561 acc 0.300 f1 0.300
[W14] GRU Epoch 11 | train_loss 1.0165 a

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W14] LSTM Epoch 01 | train_loss 1.0989 acc 0.345 f1 0.260 || val_loss 1.1023 acc 0.267 f1 0.209
[W14] LSTM Epoch 02 | train_loss 1.0946 acc 0.394 f1 0.344 || val_loss 1.1014 acc 0.312 f1 0.286
[W14] LSTM Epoch 03 | train_loss 1.0901 acc 0.413 f1 0.384 || val_loss 1.1025 acc 0.287 f1 0.286
[W14] LSTM Epoch 04 | train_loss 1.0860 acc 0.413 f1 0.406 || val_loss 1.1052 acc 0.296 f1 0.294
[W14] LSTM Epoch 05 | train_loss 1.0798 acc 0.423 f1 0.407 || val_loss 1.1101 acc 0.304 f1 0.303
[W14] LSTM Epoch 06 | train_loss 1.0698 acc 0.419 f1 0.411 || val_loss 1.1180 acc 0.300 f1 0.297
[W14] LSTM Epoch 07 | train_loss 1.0599 acc 0.433 f1 0.424 || val_loss 1.1304 acc 0.300 f1 0.292
[W14] LSTM Epoch 08 | train_loss 1.0470 acc 0.460 f1 0.457 || val_loss 1.1337 acc 0.287 f1 0.288
[W14] LSTM Epoch 09 | train_loss 1.0351 acc 0.480 f1 0.477 || val_loss 1.1532 acc 0.275 f1 0.273
[W14] LSTM Epoch 10 | train_loss 1.0249 acc 0.492 f1 0.492 || val_loss 1.1609 acc 0.304 f1 0.293
[W14] LSTM Epoch 11 | train_lo

In [17]:
"""
experiment_runner_smote.py

PyTorch experiment runner with SMOTE augmentation:

- builds sliding windows for window_sizes = [3,4,5,6,7,10,14]
- creates ANN and sequence inputs
- applies SMOTE on training data to balance classes
- trains ANN, CNN1D, RNN, GRU, LSTM
- participant-wise split (train/val/test)
- saves results to CSV, confusion matrix PNGs, and class distributions

Usage:
    python experiment_runner_smote.py
"""

import os
import copy
import random
from pathlib import Path
from collections import Counter
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, precision_recall_fscore_support, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# -------------------------
# Config
# -------------------------
CSV_PATH = "stress_detection.csv"
OUTPUT_DIR = Path("LOSO_outputs1")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
WINDOW_SIZES = [3]
RAW_FEATURES = [
    'Openness','Conscientiousness','Extraversion','Agreeableness','Neuroticism',
    'sleep_time','wake_time','sleep_duration','PSQI_score',
    'call_duration','num_calls','num_sms',
    'screen_on_time','skin_conductance','accelerometer',
    'mobility_radius','mobility_distance'
]
STATIC_PERSONALITY = ['Openness','Conscientiousness','Extraversion','Agreeableness','Neuroticism']
TARGET_COL = 'PSS_score'
CLASS_COL = 'stress_class'

TRAIN_P = 0.7
VAL_P = 0.15
TEST_P = 0.15

EPOCHS = 60
BATCH_SIZE = 64
LR = 1e-3
PATIENCE = 8
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

MODEL_NAMES = ["ANN","CNN1D","RNN","GRU","LSTM"]

# -------------------------
# Helpers
# -------------------------
def stress_to_class(score):
    s = float(score)
    if s <= 13: return 0
    if s <= 26: return 1
    return 2

def ensure_dir(path):
    Path(path).mkdir(parents=True, exist_ok=True)

# -------------------------
# Window builder
# -------------------------
def build_windows(df, raw_features, window_size=4):
    X_seq, X_flat, y, participants = [], [], [], []

    for pid, df_p in df.groupby("participant_id"):
        df_p = df_p.sort_values("day").reset_index(drop=True)
        n = len(df_p)
        if n <= window_size:
            continue
        arr = df_p[raw_features].values.astype(float)
        for i in range(n - window_size):
            window = arr[i:i+window_size]
            mean_vals = window.mean(axis=0)
            std_vals = window.std(axis=0)
            min_vals = window.min(axis=0)
            max_vals = window.max(axis=0)
            slope_vals = window[-1] - window[0]
            agg = np.concatenate([mean_vals, std_vals, min_vals, max_vals, slope_vals])

            flat = np.concatenate([window.flatten(), agg])
            target_score = df_p[TARGET_COL].iloc[i+window_size]
            cls = stress_to_class(target_score)

            X_seq.append(window.copy())
            X_flat.append(flat.copy())
            y.append(int(cls))
            participants.append(pid)

    return np.array(X_seq), np.array(X_flat), np.array(y), np.array(participants)

# -------------------------
# Dataset
# -------------------------
class StressDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

# -------------------------
# Models
# -------------------------
class ANNModel(nn.Module):
    def __init__(self, in_dim, hidden=[256,128,64], num_classes=3, p_drop=0.3):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            layers.append(nn.Linear(last,h))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(h))
            layers.append(nn.Dropout(p_drop))
            last = h
        layers.append(nn.Linear(last,num_classes))
        self.net = nn.Sequential(*layers)
    def forward(self,x): return self.net(x)

class CNN1DModel(nn.Module):
    def __init__(self, seq_len, feat_dim, num_classes=3):
        super().__init__()
        self.conv1 = nn.Conv1d(feat_dim,64,2)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64,128,2)
        self.bn2 = nn.BatchNorm1d(128)
        l = seq_len - (2-1) - (2-1)
        if l<1: l=1
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*1,64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64,num_classes)
        )
    def forward(self,x):
        x = x.permute(0,2,1)
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.gap(x)
        return self.fc(x)

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden=64, num_layers=1, rnn_type='RNN', num_classes=3, dropout=0.2):
        super().__init__()
        if rnn_type=='RNN':
            self.rnn = nn.RNN(input_dim,hidden,num_layers,batch_first=True,nonlinearity='tanh',dropout=dropout)
        elif rnn_type=='GRU':
            self.rnn = nn.GRU(input_dim,hidden,num_layers,batch_first=True,dropout=dropout)
        elif rnn_type=='LSTM':
            self.rnn = nn.LSTM(input_dim,hidden,num_layers,batch_first=True,dropout=dropout)
        else: raise ValueError("Unknown rnn_type")
        self.head = nn.Sequential(nn.Linear(hidden,32), nn.ReLU(), nn.Dropout(0.2), nn.Linear(32,num_classes))
        self.rnn_type = rnn_type
    def forward(self,x):
        out,_ = self.rnn(x)
        return self.head(out[:,-1,:])

# -------------------------
# Training & Evaluation
# -------------------------
def compute_class_weights(y):
    counts = Counter(y.tolist())
    total = sum(counts.values())
    num_classes = len(counts)
    weights = []
    for i in range(max(counts.keys())+1):
        cnt = counts.get(i,0)
        weights.append(total/(num_classes*cnt) if cnt>0 else 0.0)
    return torch.tensor(weights,dtype=torch.float32,device=DEVICE)

def train_one_epoch(model, loader, opt, criterion):
    model.train()
    total_loss = 0
    preds, trues = [], []
    for xb,yb in loader:
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        out = model(xb)
        loss = criterion(out,yb)
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item()*xb.size(0)
        preds.append(out.detach().argmax(1).cpu().numpy())
        trues.append(yb.cpu().numpy())
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    return total_loss/len(trues), accuracy_score(trues,preds), f1_score(trues,preds,average='macro',zero_division=0)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss=0
    preds,trues=[],[]
    with torch.no_grad():
        for xb,yb in loader:
            xb,yb = xb.to(DEVICE), yb.to(DEVICE)
            out = model(xb)
            total_loss += criterion(out,yb).item()*xb.size(0)
            preds.append(out.argmax(1).cpu().numpy())
            trues.append(yb.cpu().numpy())
    preds = np.concatenate(preds)
    trues = np.concatenate(trues)
    mean_loss = total_loss / len(trues)
    acc = accuracy_score(trues,preds)
    f1 = f1_score(trues,preds,average='macro',zero_division=0)
    prec, rec, f1s, sup = precision_recall_fscore_support(trues,preds,average=None,zero_division=0)
    return mean_loss, acc, f1, prec, rec, f1s, sup, preds, trues

# -------------------------
# Experiment for a window
# -------------------------
def run_experiment_for_window(df, window_size, model_name, train_pids, val_pids, test_pids):
    X_seq, X_flat, y, pids = build_windows(df, RAW_FEATURES, window_size)
    if len(y)==0: return None

    # split by participant
    train_mask = np.isin(pids, train_pids)
    val_mask = np.isin(pids, val_pids)
    test_mask = np.isin(pids, test_pids)

    X_seq_train, X_seq_val, X_seq_test = X_seq[train_mask], X_seq[val_mask], X_seq[test_mask]
    X_flat_train, X_flat_val, X_flat_test = X_flat[train_mask], X_flat[val_mask], X_flat[test_mask]
    y_train, y_val, y_test = y[train_mask], y[val_mask], y[test_mask]

    if len(y_train)==0 or len(y_val)==0 or len(y_test)==0:
        print("One split empty for window", window_size)
        return None

    # -------------------------
    # SMOTE augmentation (training set only)
    # -------------------------
    print(f"Class distribution BEFORE SMOTE: {Counter(y_train)}")
    smote = SMOTE(random_state=SEED)
    if model_name=="ANN":
        X_train_aug, y_train_aug = smote.fit_resample(X_flat_train, y_train)
        X_train_aug = X_train_aug
    else:
        N,L,F = X_seq_train.shape
        X_seq_flat = X_seq_train.reshape(N,L*F)
        X_train_aug, y_train_aug = smote.fit_resample(X_seq_flat, y_train)
        X_train_aug = X_train_aug.reshape(-1,L,F)
    print(f"Class distribution AFTER SMOTE: {Counter(y_train_aug)}")

    # save class distributions
    dist_df = pd.DataFrame({'Class':[0,1,2],
                            'Before': [Counter(y_train)[0],Counter(y_train)[1],Counter(y_train)[2]],
                            'After': [Counter(y_train_aug)[0],Counter(y_train_aug)[1],Counter(y_train_aug)[2]]})
    dist_df.to_csv(OUTPUT_DIR/f'class_distribution_w{window_size}_{model_name}.csv', index=False)

    # -------------------------
    # Scaling
    # -------------------------
    if model_name=="ANN":
        scaler = StandardScaler().fit(X_train_aug)
        Xtr = scaler.transform(X_train_aug)
        Xv = scaler.transform(X_flat_val)
        Xt = scaler.transform(X_flat_test)
        input_dim = Xtr.shape[1]
    else:
        N,L,F = X_train_aug.shape
        scaler = StandardScaler().fit(X_train_aug.reshape(-1,F))
        Xtr = scaler.transform(X_train_aug.reshape(-1,F)).reshape(N,L,F)
        Xv = scaler.transform(X_seq_val.reshape(-1,F)).reshape(X_seq_val.shape)
        Xt = scaler.transform(X_seq_test.reshape(-1,F)).reshape(X_seq_test.shape)
        input_dim = F

    train_ds = StressDataset(Xtr, y_train_aug)
    val_ds = StressDataset(Xv, y_val)
    test_ds = StressDataset(Xt, y_test)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    # -------------------------
    # Model
    # -------------------------
    if model_name=="ANN": model = ANNModel(input_dim).to(DEVICE)
    elif model_name=="CNN1D": model = CNN1DModel(Xtr.shape[1], Xtr.shape[2]).to(DEVICE)
    elif model_name in ("RNN","GRU","LSTM"): model = RNNModel(input_dim, hidden=64, num_layers=1, rnn_type=model_name).to(DEVICE)
    else: raise ValueError("Unknown model")

    # loss and optimizer
    class_weights = compute_class_weights(torch.tensor(y_train_aug))
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)

    # -------------------------
    # Training loop with early stopping
    # -------------------------
    best_val_f1, best_state, cur_wait = -1.0, None, 0
    for epoch in range(1,EPOCHS+1):
        train_loss, train_acc, train_f1 = train_one_epoch(model, train_loader, opt, criterion)
        val_loss, val_acc, val_f1, *_ = evaluate(model, val_loader, criterion)
        print(f"[W{window_size}] {model_name} Epoch {epoch:02d} | train_loss {train_loss:.4f} acc {train_acc:.3f} f1 {train_f1:.3f} || val_loss {val_loss:.4f} acc {val_acc:.3f} f1 {val_f1:.3f}")
        if val_f1 > best_val_f1+1e-4:
            best_val_f1 = val_f1
            best_state = copy.deepcopy(model.state_dict())
            cur_wait = 0
        else:
            cur_wait += 1
            if cur_wait >= PATIENCE:
                print("Early stopping.")
                break

    # test
    if best_state is not None: model.load_state_dict(best_state)
    test_loss, test_acc, test_f1, prec, rec, f1s, sup, preds, trues = evaluate(model, test_loader, criterion)

    # save confusion matrix
    cm = confusion_matrix(trues,preds)
    np.save(OUTPUT_DIR/f"cm_w{window_size}_{model_name}.npy", cm)
    plt.figure(figsize=(4,4))
    plt.imshow(cm, cmap='Blues')
    plt.title(f"CM W{window_size} {model_name}")
    plt.colorbar()
    plt.xlabel("pred")
    plt.ylabel("true")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR/f"cm_w{window_size}_{model_name}.png")
    plt.close()

    return {
        "window": window_size,
        "model": model_name,
        "train_samples": len(y_train_aug),
        "val_samples": len(y_val),
        "test_samples": len(y_test),
        "test_loss": float(test_loss),
        "test_acc": float(test_acc),
        "test_macro_f1": float(test_f1),
        "per_class_prec": prec.tolist(),
        "per_class_rec": rec.tolist(),
        "per_class_f1": f1s.tolist(),
        "support": sup.tolist()
    }

# -------------------------
# Main
# -------------------------
def main():
    print("Device:", DEVICE)
    df = pd.read_csv(CSV_PATH)
    df = df.sort_values(['participant_id','day']).reset_index(drop=True)
    df[CLASS_COL] = df[TARGET_COL].apply(stress_to_class)

    # participant split
    pids = df['participant_id'].unique()
    random.shuffle(pids)
    n = len(pids)
    n_train = int(n*TRAIN_P)
    n_val = int(n*VAL_P)
    train_pids = pids[:n_train]
    val_pids = pids[n_train:n_train+n_val]
    test_pids = pids[n_train+n_val:]
    print(f"Participants: total {n} train {len(train_pids)} val {len(val_pids)} test {len(test_pids)}")

    results=[]
    for w in WINDOW_SIZES:
        for mname in MODEL_NAMES:
            print("="*60)
            print(f"Running Window={w} Model={mname}")
            try:
                res = run_experiment_for_window(df, w, mname, train_pids, val_pids, test_pids)
                if res is not None:
                    results.append(res)
                    pd.DataFrame(results).to_csv(OUTPUT_DIR/"results_summary.csv", index=False)
            except Exception as e:
                print("Error running", w, mname, e)

    if results:
        pd.DataFrame(results).to_csv(OUTPUT_DIR/"results_summary.csv", index=False)
        print("Saved results to", OUTPUT_DIR/"results_summary.csv")
    else:
        print("No results to save.")

# -------------------------
# Main LOSO
# -------------------------
def main_loso():
    print("Device:", DEVICE)
    df = pd.read_csv(CSV_PATH)
    df = df.sort_values(['participant_id','day']).reset_index(drop=True)
    df[CLASS_COL] = df[TARGET_COL].apply(stress_to_class)

    pids = df['participant_id'].unique()
    results = []

    for test_pid in tqdm(pids, desc="LOSO participants"):
        train_val_pids = [pid for pid in pids if pid != test_pid]

        # Optional: split train_val_pids into train/val
        n_train = int(len(train_val_pids) * (TRAIN_P/(TRAIN_P+VAL_P)))
        train_pids = train_val_pids[:n_train]
        val_pids = train_val_pids[n_train:]

        print(f"LOSO: Test subject={test_pid} | Train subjects={len(train_pids)} | Val subjects={len(val_pids)}")

        for w in WINDOW_SIZES:
            for mname in MODEL_NAMES:
                print("="*60)
                print(f"Running Window={w} Model={mname} Test subject={test_pid}")
                try:
                    res = run_experiment_for_window(df, w, mname, train_pids, val_pids, [test_pid])
                    if res is not None:
                        res["held_out_subject"] = test_pid
                        results.append(res)
                        # save intermediate results
                        pd.DataFrame(results).to_csv(OUTPUT_DIR/"results_loso.csv", index=False)
                except Exception as e:
                    print("Error running", w, mname, "for subject", test_pid, e)

    if results:
        pd.DataFrame(results).to_csv(OUTPUT_DIR/"results_loso.csv", index=False)
        print("Saved LOSO results to", OUTPUT_DIR/"results_loso.csv")
    else:
        print("No results to save.")

if __name__=="__main__":
    main_loso()



Device: cpu


LOSO participants:   0%|          | 0/100 [00:00<?, ?it/s]

LOSO: Test subject=1 | Train subjects=81 | Val subjects=18
Running Window=3 Model=ANN Test subject=1
Class distribution BEFORE SMOTE: Counter({np.int64(2): 984, np.int64(1): 932, np.int64(0): 271})
Class distribution AFTER SMOTE: Counter({np.int64(0): 984, np.int64(2): 984, np.int64(1): 984})
[W3] ANN Epoch 01 | train_loss 1.1128 acc 0.410 f1 0.407 || val_loss 1.1229 acc 0.331 f1 0.314
[W3] ANN Epoch 02 | train_loss 0.9835 acc 0.497 f1 0.490 || val_loss 1.1312 acc 0.364 f1 0.338
[W3] ANN Epoch 03 | train_loss 0.8912 acc 0.541 f1 0.533 || val_loss 1.1224 acc 0.344 f1 0.314
[W3] ANN Epoch 04 | train_loss 0.8094 acc 0.597 f1 0.589 || val_loss 1.1153 acc 0.368 f1 0.317
[W3] ANN Epoch 05 | train_loss 0.7454 acc 0.643 f1 0.637 || val_loss 1.1263 acc 0.379 f1 0.317
[W3] ANN Epoch 06 | train_loss 0.7153 acc 0.644 f1 0.640 || val_loss 1.1393 acc 0.370 f1 0.323
[W3] ANN Epoch 07 | train_loss 0.6591 acc 0.684 f1 0.681 || val_loss 1.1263 acc 0.379 f1 0.320
[W3] ANN Epoch 08 | train_loss 0.6248 acc

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W3] RNN Epoch 02 | train_loss 1.0825 acc 0.419 f1 0.412 || val_loss 1.1021 acc 0.311 f1 0.296
[W3] RNN Epoch 03 | train_loss 1.0632 acc 0.444 f1 0.438 || val_loss 1.0945 acc 0.346 f1 0.321
[W3] RNN Epoch 04 | train_loss 1.0465 acc 0.454 f1 0.448 || val_loss 1.0884 acc 0.337 f1 0.305
[W3] RNN Epoch 05 | train_loss 1.0242 acc 0.480 f1 0.474 || val_loss 1.0860 acc 0.354 f1 0.317
[W3] RNN Epoch 06 | train_loss 1.0050 acc 0.496 f1 0.489 || val_loss 1.0728 acc 0.389 f1 0.345
[W3] RNN Epoch 07 | train_loss 0.9860 acc 0.518 f1 0.511 || val_loss 1.1041 acc 0.337 f1 0.310
[W3] RNN Epoch 08 | train_loss 0.9631 acc 0.532 f1 0.523 || val_loss 1.0902 acc 0.370 f1 0.323
[W3] RNN Epoch 09 | train_loss 0.9432 acc 0.548 f1 0.537 || val_loss 1.1128 acc 0.350 f1 0.321
[W3] RNN Epoch 10 | train_loss 0.9215 acc 0.563 f1 0.553 || val_loss 1.1245 acc 0.350 f1 0.317
[W3] RNN Epoch 11 | train_loss 0.9064 acc 0.565 f1 0.554 || val_loss 1.1155 acc 0.364 f1 0.331
[W3] RNN Epoch 12 | train_loss 0.8745 acc 0.587 f1

...

[W3] GRU Epoch 01 | train_loss 1.1031 acc 0.342 f1 0.227 || val_loss 1.1106 acc 0.276 f1 0.261
[W3] GRU Epoch 02 | train_loss 1.0899 acc 0.401 f1 0.398 || val_loss 1.0961 acc 0.335 f1 0.313
[W3] GRU Epoch 03 | train_loss 1.0791 acc 0.421 f1 0.416 || val_loss 1.0973 acc 0.335 f1 0.318
[W3] GRU Epoch 04 | train_loss 1.0630 acc 0.445 f1 0.443 || val_loss 1.1047 acc 0.319 f1 0.307
[W3] GRU Epoch 05 | train_loss 1.0327 acc 0.476 f1 0.470 || val_loss 1.1036 acc 0.321 f1 0.306
[W3] GRU Epoch 06 | train_loss 0.9900 acc 0.519 f1 0.509 || val_loss 1.0642 acc 0.377 f1 0.318
[W3] GRU Epoch 07 | train_loss 0.9253 acc 0.561 f1 0.555 || val_loss 1.1260 acc 0.319 f1 0.300
[W3] GRU Epoch 08 | train_loss 0.8496 acc 0.593 f1 0.587 || val_loss 1.1309 acc 0.352 f1 0.316
[W3] GRU Epoch 09 | train_loss 0.7919 acc 0.613 f1 0.606 || val_loss 1.1173 acc 0.385 f1 0.326
[W3] GRU Epoch 10 | train_loss 0.7456 acc 0.644 f1 0.639 || val_loss 1.1220 acc 0.403 f1 0.349
[W3] GRU Epoch 11 | train_loss 0.7124 acc 0.659 f1

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[W3] LSTM Epoch 01 | train_loss 1.1019 acc 0.335 f1 0.174 || val_loss 1.0909 acc 0.465 f1 0.215
[W3] LSTM Epoch 02 | train_loss 1.0947 acc 0.366 f1 0.348 || val_loss 1.0967 acc 0.377 f1 0.340
[W3] LSTM Epoch 03 | train_loss 1.0887 acc 0.412 f1 0.409 || val_loss 1.1032 acc 0.323 f1 0.304
[W3] LSTM Epoch 04 | train_loss 1.0779 acc 0.434 f1 0.432 || val_loss 1.0921 acc 0.350 f1 0.319
[W3] LSTM Epoch 05 | train_loss 1.0491 acc 0.467 f1 0.463 || val_loss 1.1119 acc 0.298 f1 0.288
[W3] LSTM Epoch 06 | train_loss 0.9723 acc 0.527 f1 0.514 || val_loss 1.0775 acc 0.389 f1 0.314
[W3] LSTM Epoch 07 | train_loss 0.8810 acc 0.572 f1 0.566 || val_loss 1.1098 acc 0.364 f1 0.299
[W3] LSTM Epoch 08 | train_loss 0.8195 acc 0.611 f1 0.607 || val_loss 1.1650 acc 0.385 f1 0.326
[W3] LSTM Epoch 09 | train_loss 0.7784 acc 0.632 f1 0.627 || val_loss 1.1506 acc 0.401 f1 0.329
[W3] LSTM Epoch 10 | train_loss 0.7286 acc 0.654 f1 0.652 || val_loss 1.2175 acc 0.377 f1 0.310
Early stopping.


LOSO participants: 100%|██████████| 100/100 [47:47<00:00, 28.67s/it]

Saved LOSO results to LOSO_outputs1/results_loso.csv


In [20]:
import pandas as pd
from pathlib import Path

# Path to LOSO results CSV
INPUT_FILE = Path("LOSO_outputs1/results_loso.csv")
OUTPUT_FILE = Path("LOSO_outputs1/results_loso_avg.csv")

# Load LOSO results
if not INPUT_FILE.exists():
    raise FileNotFoundError(f"{INPUT_FILE} not found!")

df = pd.read_csv(INPUT_FILE)

# Columns to average
metrics = ['test_loss', 'test_acc', 'test_macro_f1']

# Group by model and compute mean and standard deviation
avg_df = df.groupby('model')[metrics].agg(['mean', 'std']).reset_index()

# Flatten MultiIndex columns
avg_df.columns = ['_'.join(col).strip('_') for col in avg_df.columns.values]

# Save to CSV
avg_df.to_csv(OUTPUT_FILE, index=False)
print(f"Saved LOSO averages per model to {OUTPUT_FILE}")

# Optional: print summary
print("\nAverage LOSO performance per model:")
print(avg_df)


Saved LOSO averages per model to LOSO_outputs1/results_loso_avg.csv

Average LOSO performance per model:
   model  test_loss_mean  test_loss_std  test_acc_mean  test_acc_std  \
0    ANN        1.366389       0.304318       0.390741      0.094518   
1  CNN1D        1.531317       0.606295       0.400000      0.091785   
2    GRU        1.230809       0.369776       0.381111      0.093918   
3   LSTM        1.299862       0.397285       0.371481      0.088941   
4    RNN        1.183072       0.247508       0.362222      0.089116   

   test_macro_f1_mean  test_macro_f1_std  
0            0.303756           0.087204  
1            0.320086           0.086321  
2            0.315348           0.086713  
3            0.306517           0.079003  
4            0.308841           0.087930  
